In [ ]:
# unverified
def get_IIT_arithmetic_dataset_control_V1(
    variable_range, seed,
    data_size,
    combiner_func
):
    base, base_y, source, source_y = get_IIT_arithmetic_dataset_factual_pairs(
        variable_range, seed,
        data_size,
        combiner_func
    )
    
    counterfactual_y = []
    for i in range(len(base)):
        c_y = combiner_func(source[i][0], base[i][1], base[i][-1])
        counterfactual_y += [c_y]
        
    return torch.tensor(base, dtype=torch.long), \
        torch.tensor(base_y, dtype=torch.long), \
        torch.tensor(source, dtype=torch.long), \
        torch.tensor(counterfactual_y, dtype=torch.long), \
        torch.tensor([2 for _ in range(len(base))], dtype=torch.long)

def get_IIT_arithmetic_dataset_control_V2(
    variable_range, seed,
    data_size,
    combiner_func
):
    base, base_y, source, source_y = get_IIT_arithmetic_dataset_factual_pairs(
        variable_range, seed,
        data_size,
        combiner_func
    )
    
    counterfactual_y = []
    for i in range(len(base)):
        c_y = combiner_func(base[i][0], source[i][1], base[i][-1])
        counterfactual_y += [c_y]
        
    return torch.tensor(base, dtype=torch.long), \
        torch.tensor(base_y, dtype=torch.long), \
        torch.tensor(source, dtype=torch.long), \
        torch.tensor(counterfactual_y, dtype=torch.long), \
        torch.tensor([2 for _ in range(len(base))], dtype=torch.long)

def get_IIT_arithmetic_dataset_sum_first_V2(
    variable_range, seed,
    data_size,
    combiner_func
):
    base, base_y, source, source_y = get_IIT_arithmetic_dataset_factual_pairs(
        variable_range, seed,
        data_size,
        combiner_func
    )
    
    counterfactual_y = []
    for i in range(len(base)):
        c_y = (base[i][0]+base[i][1])*source[i][-1]
        counterfactual_y += [c_y]
        
    return torch.tensor(base, dtype=torch.long), \
        torch.tensor(base_y, dtype=torch.long), \
        torch.tensor(source, dtype=torch.long), \
        torch.tensor(counterfactual_y, dtype=torch.long), \
        torch.tensor([1 for _ in range(len(base))], dtype=torch.long)

def get_IIT_arithmetic_dataset_prod_first_V1(
    variable_range, seed,
    data_size,
    combiner_func
):
    base, base_y, source, source_y = get_IIT_arithmetic_dataset_factual_pairs(
        variable_range, seed,
        data_size,
        combiner_func
    )
    
    counterfactual_y = []
    for i in range(len(base)):
        c_y = (base[i][1]*base[i][-1])+(source[i][0]*source[i][-1])
        counterfactual_y += [c_y]
        
    return torch.tensor(base, dtype=torch.long), \
        torch.tensor(base_y, dtype=torch.long), \
        torch.tensor(source, dtype=torch.long), \
        torch.tensor(counterfactual_y, dtype=torch.long), \
        torch.tensor([0 for _ in range(len(base))], dtype=torch.long)

def get_IIT_arithmetic_dataset_prod_first_V2(
    variable_range, seed,
    data_size,
    combiner_func
):
    base, base_y, source, source_y = get_IIT_arithmetic_dataset_factual_pairs(
        variable_range, seed,
        data_size,
        combiner_func
    )
    
    counterfactual_y = []
    for i in range(len(base)):
        c_y = (base[i][0]*base[i][-1])+(source[i][1]*source[i][-1])
        counterfactual_y += [c_y]
        
    return torch.tensor(base, dtype=torch.long), \
        torch.tensor(base_y, dtype=torch.long), \
        torch.tensor(source, dtype=torch.long), \
        torch.tensor(counterfactual_y, dtype=torch.long), \
        torch.tensor([1 for _ in range(len(base))], dtype=torch.long)

def get_IIT_arithmetic_dataset_control(
    variable_range, seed,
    data_size,
    combiner_func
):
    base, base_y, source, source_y = get_IIT_arithmetic_dataset_factual_pairs(
        variable_range, seed,
        data_size,
        combiner_func
    )
    
    counterfactual_y = []
    for i in range(len(base)):
        c_y = source_y[i]
        counterfactual_y += [c_y]
        
    return torch.tensor(base, dtype=torch.long), \
        torch.tensor(base_y, dtype=torch.long), \
        torch.tensor(source, dtype=torch.long), \
        torch.tensor(counterfactual_y, dtype=torch.long), \
        torch.tensor([2 for _ in range(len(base))], dtype=torch.long)

def get_IIT_arithmetic_dataset_sum_first_V1(
    variable_range, seed,
    data_size,
    combiner_func
):
    base, base_y, source, source_y = get_IIT_arithmetic_dataset_factual_pairs(
        variable_range, seed,
        data_size,
        combiner_func
    )
    
    source_sum = []
    for s in source:
        s_sum = s[0]+s[1]
        source_sum += [s_sum]
    
    counterfactual_y = []
    for i in range(len(base)):
        c_y = source_sum[i]*base[i][-1]
        counterfactual_y += [c_y]
        
    return torch.tensor(base, dtype=torch.long), \
        torch.tensor(base_y, dtype=torch.long), \
        torch.tensor(source, dtype=torch.long), \
        torch.tensor(counterfactual_y, dtype=torch.long), \
        torch.tensor([0 for _ in range(len(base))], dtype=torch.long)


In [33]:
from random import randrange
import torch, gc
import random
import copy
import itertools
import numpy as np
import utils
from trainer import LIMTrainer
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
from LIM_deep_neural_classifier import LIMDeepNeuralClassifier
import dataset_equality

gc.collect()
torch.cuda.empty_cache()
utils.fix_random_seeds()

def ab_c(a, b, c):
    return (a+b)*c

def get_IIT_arithmetic_dataset_factuals(
    variable_range, seed,
    data_size,
    combiner_func
):
    random.seed(seed)
    base = [[random.randint(
        variable_range[0], variable_range[1]
    ) for _ in range(3)] for _ in range(data_size)]
    base_y = []
    for b in base:
        b_y = combiner_func(*b)
        base_y += [b_y]
        
    return torch.tensor(base, dtype=torch.long), torch.tensor(base_y, dtype=torch.long)

def get_IIT_arithmetic_dataset_factual_pairs(
    variable_range, seed,
    data_size,
    combiner_func
):
    random.seed(seed)
    base = [[random.randint(
        variable_range[0], variable_range[1]
    ) for _ in range(3)] for _ in range(data_size)]
    base_y = []
    for b in base:
        b_y = combiner_func(*b)
        base_y += [b_y]
        
    source = [[random.randint(
        variable_range[0], variable_range[1]
    ) for _ in range(3)] for _ in range(data_size)]
    source_y = []
    for s in source:
        s_y = combiner_func(*s)
        source_y += [s_y]
        
    return base, \
        base_y, \
        source, \
        source_y

def get_IIT_arithmetic_dataset_sum_first_V1(
    variable_range, seed,
    data_size,
    combiner_func
):
    base, base_y, source, source_y = get_IIT_arithmetic_dataset_factual_pairs(
        variable_range, seed,
        data_size,
        combiner_func
    )
    
    source_sum = []
    for s in source:
        s_sum = s[0]+s[1]
        source_sum += [s_sum]
    
    counterfactual_y = []
    for i in range(len(base)):
        c_y = source_sum[i]*base[i][-1]
        counterfactual_y += [c_y]
        
    return torch.tensor(base, dtype=torch.long), \
        torch.tensor(base_y, dtype=torch.long), \
        [torch.tensor(source, dtype=torch.long), torch.tensor(source, dtype=torch.long)], \
        torch.tensor(counterfactual_y, dtype=torch.long), \
        torch.tensor([0 for _ in range(len(base))], dtype=torch.long)

def get_IIT_arithmetic_dataset_sum_first_V2(
    variable_range, seed,
    data_size,
    combiner_func
):
    base, base_y, source, source_y = get_IIT_arithmetic_dataset_factual_pairs(
        variable_range, seed,
        data_size,
        combiner_func
    )
    
    counterfactual_y = []
    for i in range(len(base)):
        c_y = (base[i][0]+base[i][1])*source[i][-1]
        counterfactual_y += [c_y]
        
    return torch.tensor(base, dtype=torch.long), \
        torch.tensor(base_y, dtype=torch.long), \
        [torch.tensor(source, dtype=torch.long), torch.tensor(source, dtype=torch.long)], \
        torch.tensor(counterfactual_y, dtype=torch.long), \
        torch.tensor([1 for _ in range(len(base))], dtype=torch.long)

def get_IIT_arithmetic_dataset_sum_first_V1_V2(
    variable_range, seed,
    data_size,
    combiner_func
):
    base_1, base_y_1, source_1, source_y_1 = get_IIT_arithmetic_dataset_factual_pairs(
        variable_range, seed,
        data_size,
        combiner_func
    )
    base_2, base_y_2, source_2, source_y_2 = get_IIT_arithmetic_dataset_factual_pairs(
        variable_range, seed+1,
        data_size,
        combiner_func
    )
    
    source_sum_1 = []
    for s in source_1:
        s_sum = s[0]+s[1]
        source_sum_1 += [s_sum]
    
    counterfactual_y = []
    for i in range(len(source_2)):
        c_y = source_sum_1[i]*source_2[i][-1]
        counterfactual_y += [c_y]
    
    return torch.tensor(base_1, dtype=torch.long), \
        torch.tensor(base_y_1, dtype=torch.long), \
        [torch.tensor(source_1, dtype=torch.long), torch.tensor(source_2, dtype=torch.long)], \
        torch.tensor(counterfactual_y, dtype=torch.long), \
        torch.tensor([2 for _ in range(len(base_1))], dtype=torch.long)

def get_IIT_arithmetic_dataset_sum_first(
    variable_range, seed,
    data_size,
    combiner_func
):
    V1_dataset = get_IIT_arithmetic_dataset_sum_first_V1(
        variable_range, seed,
        data_size,
        combiner_func
    )
    V2_dataset = get_IIT_arithmetic_dataset_sum_first_V2(
        variable_range, seed,
        data_size,
        combiner_func
    )
    both_dataset = get_IIT_arithmetic_dataset_sum_first_V1_V2(
        variable_range, seed,
        data_size,
        combiner_func
    )
    combined_dataset = [torch.cat((V1_dataset[0],
                                    V2_dataset[0],
                                    both_dataset[0])),
                       torch.cat((V1_dataset[1],
                                    V2_dataset[1],
                                    both_dataset[1])),
                       [torch.cat((V1_dataset[2][0],
                                    V2_dataset[2][0],
                                    both_dataset[2][0])),
                       torch.cat((V1_dataset[2][0],
                                    V2_dataset[2][0],
                                    both_dataset[2][1]))],
                       torch.cat((V1_dataset[3],
                                    V2_dataset[3],
                                    both_dataset[3])),
                       torch.cat((V1_dataset[4],
                                    V2_dataset[4],
                                    both_dataset[4]))]
    return combined_dataset

### Train Factual Models

In [3]:
# Fixed variables
device = "cpu"
data_size = 6400
num_layers = 3
max_iter = 500
variable_range = [1, 6]

embedding_dim = variable_range[1]-variable_range[0]+1
input_dim = embedding_dim * 3
min_y = (variable_range[0]+variable_range[0])*variable_range[0]
max_y = (variable_range[1]+variable_range[1])*variable_range[1]
classes = sorted(set([i for i in range(min_y, max_y+1)]))
n_classes = (variable_range[1]+variable_range[1])*variable_range[1]
class2index = dict(zip(classes, range(n_classes)))

In [4]:
for seed in {42, 77, 88}:
    for hidden_dim in {18, 36}: # {18, 36}
        utils.fix_random_seeds(seed=seed)
        print(f"training factual model for seed={seed}")
        X_base_train, y_base_train = get_IIT_arithmetic_dataset_factuals(
            variable_range, seed, data_size, ab_c
        )
        LIM = LIMDeepNeuralClassifier(
            hidden_dim=hidden_dim, 
            hidden_activation=torch.nn.ReLU(), 
            num_layers=num_layers,
            input_dim=input_dim,
            n_classes=n_classes,
            device=device,
            vocab_size=variable_range[1]-variable_range[0]+1,
            embed_dim=embedding_dim,
        )
        
        LIM_trainer = LIMTrainer(
            LIM,
            warm_start=True,
            max_iter=max_iter,
            batch_size=6400,
            n_iter_no_change=10000,
            shuffle_train=False,
            eta=0.01,
            input_as_ids=True,
            device=device,
            class2index=class2index,
            save_checkpoint_per_epoch=True,
            seed=seed
        )
        
        _ = LIM_trainer.fit(
            X_base_train, 
            y_base_train, 
            iit_data=None,
            intervention_ids_to_coords=None
        )
        
        PATH = f"./saved_models_arithmetic/basemodel-last-{num_layers}-{hidden_dim}-{seed}.bin"
        torch.save(LIM_trainer.model.state_dict(), PATH)

training factual model for seed=88
unfreezing the embedding layer as well.


Finished epoch 2 of 500; error is 4.2484803199768073973346

training factual model for seed=88
unfreezing the embedding layer as well.


Finished epoch 1 of 500; error is 4.3115525245666517369097

training factual model for seed=42
unfreezing the embedding layer as well.


...

Finished epoch 1 of 500; error is 4.2826848030090335912848

training factual model for seed=77
unfreezing the embedding layer as well.


Finished epoch 3 of 500; error is 4.162506580352783568093

training factual model for seed=77
unfreezing the embedding layer as well.


Finished epoch 500 of 500; error is 0.00023455559858120978

### IIT Dataset Generation

### Train Oracle Models

In [ ]:
dataset_fun = get_IIT_arithmetic_dataset_sum_first
iit_data_size = 64000
iit_max_iter = 50
model_c = 1
total_model_c = 135
for seed in {42, 77, 88}: # {42, 77, 88}
    utils.fix_random_seeds(seed=seed)
    train_datasetIIT = get_IIT_arithmetic_dataset_sum_first(
        variable_range, seed, iit_data_size, ab_c
    )
    X_base_train, y_base_train = train_datasetIIT[0:2]
    iit_data = tuple(train_datasetIIT[2:])
    for hidden_dim in {18, 36}: # {18, 36}
        for hidden_dim_per_concept in {1, 2, 6}: # {1, 2, 6}
            for iit_layer in [0, 1, 2]: # [0, 1, 2]
                scale_factor = hidden_dim/9
                small_scale_factor = hidden_dim/18
                id_to_coords = {
                    0: [{"layer": iit_layer, "start": 0, "end": int(scale_factor*hidden_dim_per_concept)}],
                    1: [{"layer": iit_layer, "start":  int(scale_factor*hidden_dim_per_concept), "end": int((scale_factor+small_scale_factor)*hidden_dim_per_concept)}],
                    2: [{"layer": iit_layer, "start": 0, "end": int(scale_factor*hidden_dim_per_concept)}, 
                        {"layer": iit_layer, "start":  int(scale_factor*hidden_dim_per_concept), "end": int((scale_factor+small_scale_factor)*hidden_dim_per_concept)}],
                }
                print("id_to_coords: ", id_to_coords)
                for i in [100, 200, 300, 400, 500]:
                    print(f"Training {model_c}/{total_model_c} model aligned with oracle hlm with params:")
                    print(f"seed={seed}")
                    print(f"hidden_dim_per_concept={hidden_dim_per_concept}")
                    print(f"iit_layer={iit_layer}")
                    print(f"epoch={i}")
                    LIM = LIMDeepNeuralClassifier(
                        hidden_dim=hidden_dim, 
                        hidden_activation=torch.nn.ReLU(), 
                        num_layers=num_layers,
                        input_dim=input_dim,
                        n_classes=n_classes,
                        device=device,
                        vocab_size=variable_range[1]-variable_range[0]+1,
                        embed_dim=embedding_dim,
                    )
                    LIM_trainer = LIMTrainer(
                        LIM,
                        warm_start=True,
                        max_iter=iit_max_iter,
                        batch_size=6400,
                        n_iter_no_change=10000,
                        shuffle_train=False,
                        eta=0.01,
                        input_as_ids=True,
                        device=device,
                        class2index=class2index,
                        save_checkpoint_per_epoch=False,
                        seed=seed
                    )
        
                    PATH = f"./saved_models_arithmetic/basemodel-{i}-{num_layers}-{hidden_dim}-{seed}.bin"
                    LIM_trainer.model.load_state_dict(torch.load(PATH))
                    LIM_trainer.model.set_analysis_mode(True)

                    _ = LIM_trainer.fit(
                        X_base_train, 
                        y_base_train, 
                        iit_data=iit_data,
                        intervention_ids_to_coords=id_to_coords)
                    
                    iit_layer_out = iit_layer + 1
                    PATH = f"./saved_models_arithmetic/iit-oraclemodel-epoch{i}-"\
                           f"{iit_layer_out}-{hidden_dim}-{hidden_dim_per_concept}-"\
                           f"{seed}.bin"
                    torch.save(LIM_trainer.model.state_dict(), PATH)

                    model_c += 1
                    
                    

id_to_coords:  {0: [{'layer': 0, 'start': 0, 'end': 2}], 1: [{'layer': 0, 'start': 2, 'end': 3}], 2: [{'layer': 0, 'start': 0, 'end': 2}, {'layer': 0, 'start': 2, 'end': 3}]}
Training 1/135 model aligned with oracle hlm with params:
seed=88
hidden_dim_per_concept=1
iit_layer=0
epoch=100
unfreezing the embedding layer as well.
freezing the embedding layer as well.


Finished epoch 50 of 50; error is 411.95255422592163

Training 2/135 model aligned with oracle hlm with params:
seed=88
hidden_dim_per_concept=1
iit_layer=0
epoch=200
unfreezing the embedding layer as well.
freezing the embedding layer as well.


Finished epoch 50 of 50; error is 572.7030506134033

Training 3/135 model aligned with oracle hlm with params:
seed=88
hidden_dim_per_concept=1
iit_layer=0
epoch=300
unfreezing the embedding layer as well.
freezing the embedding layer as well.


...

Finished epoch 50 of 50; error is 216.68982219696045

Training 138/135 model aligned with oracle hlm with params:
seed=42
hidden_dim_per_concept=1
iit_layer=0
epoch=300
unfreezing the embedding layer as well.
freezing the embedding layer as well.


Finished epoch 50 of 50; error is 229.32872295379644

Training 139/135 model aligned with oracle hlm with params:
seed=42
hidden_dim_per_concept=1
iit_layer=0
epoch=400
unfreezing the embedding layer as well.
freezing the embedding layer as well.


Finished epoch 16 of 50; error is 240.49215030670166

In [54]:
1

1

### Eval Oracle Models

In [51]:
oracle_results = []
dataset_fun = get_IIT_arithmetic_dataset_sum_first
model_c = 1
total_model_c = 135
iit_test_data_size = 6400
for seed in {77}: # {42, 77, 88}
    utils.fix_random_seeds(seed=seed)
    base_test, y_base_test, sources_test, y_IIT_test, intervention_ids_test  = \
        get_IIT_arithmetic_dataset_sum_first(
            variable_range, seed+1, iit_test_data_size, ab_c
        )
    for hidden_dim in {36}: # {18, 36}
        for hidden_dim_per_concept in {6}: # {1, 2, 6}
            for iit_layer in [0]: # [0, 1, 2]
                scale_factor = hidden_dim/9
                small_scale_factor = hidden_dim/18
                id_to_coords = {
                    0: [{"layer": iit_layer, "start": 0, "end": int(scale_factor*hidden_dim_per_concept)}],
                    1: [{"layer": iit_layer, "start":  int(scale_factor*hidden_dim_per_concept), "end": int((scale_factor+small_scale_factor)*hidden_dim_per_concept)}],
                    2: [{"layer": iit_layer, "start": 0, "end": int(scale_factor*hidden_dim_per_concept)}, 
                        {"layer": iit_layer, "start":  int(scale_factor*hidden_dim_per_concept), "end": int((scale_factor+small_scale_factor)*hidden_dim_per_concept)}],
                }
                print("id_to_coords: ", id_to_coords)
                for i in [500]:
                    LIM = LIMDeepNeuralClassifier(
                        hidden_dim=hidden_dim, 
                        hidden_activation=torch.nn.ReLU(), 
                        num_layers=num_layers,
                        input_dim=input_dim,
                        n_classes=n_classes,
                        device=device,
                        vocab_size=variable_range[1]-variable_range[0]+1,
                        embed_dim=embedding_dim,
                    )
                    LIM_trainer = LIMTrainer(
                        LIM,
                        warm_start=True,
                        max_iter=max_iter,
                        batch_size=6400,
                        n_iter_no_change=10000,
                        shuffle_train=False,
                        eta=0.01,
                        input_as_ids=True,
                        device=device,
                        class2index=class2index,
                        save_checkpoint_per_epoch=False,
                        seed=seed
                    )
                    
                    iit_layer_out = iit_layer + 1
                    PATH = f"./saved_models_arithmetic/iit-oraclemodel-epoch{i}-"\
                           f"{iit_layer_out}-{hidden_dim}-{hidden_dim_per_concept}-"\
                           f"{seed}.bin"
                    LIM_trainer.model.load_state_dict(torch.load(PATH))
                    LIM_trainer.model.set_analysis_mode(True)
                    
                    # test data eval
                    base_preds = LIM_trainer.predict(
                        base_test, device="cpu"
                    )
                    IIT_preds = LIM_trainer.iit_predict(
                        base_test, sources_test, 
                        intervention_ids_test, 
                        id_to_coords, device="cpu"
                    )
                    r1 = classification_report(y_base_test, base_preds, output_dict=True)
                    r2 = classification_report(y_IIT_test, IIT_preds, output_dict=True)
                    
                    print(r1["weighted avg"]["f1-score"])
                    print(r2["weighted avg"]["f1-score"])
                    

id_to_coords:  {0: [{'layer': 0, 'start': 0, 'end': 24}], 1: [{'layer': 0, 'start': 24, 'end': 36}], 2: [{'layer': 0, 'start': 0, 'end': 24}, {'layer': 0, 'start': 24, 'end': 36}]}
unfreezing the embedding layer as well.
freezing the embedding layer as well.
1.0
0.5389900169054678


In [234]:
X_base_test, y_base_test, X_sources_test, y_IIT_test, intervention_ids_test = \
    dataset_fun(
    variable_range, 42, 1000, ab_c
)

In [237]:
_ = LIM_trainer.fit(
    X_base_train, 
    y_base_train, 
    iit_data=iit_data,
    intervention_ids_to_coords=id_to_coords)

Finished epoch 500 of 500; error is 2.2967379093170166

In [238]:
X_base_test, y_base_test, X_sources_test, y_IIT_test, intervention_ids_test = \
    dataset_fun(
    variable_range, 42, 1000, ab_c
)

/afs/cs.stanford.edu/u/wuzhengx/.local/lib/python3.7/site-packages/ipykernel_launcher.py:87: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/afs/cs.stanford.edu/u/wuzhengx/.local/lib/python3.7/site-packages/ipykernel_launcher.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/afs/cs.stanford.edu/u/wuzhengx/.local/lib/python3.7/site-packages/ipykernel_launcher.py:89: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [239]:
IIT_preds = LIM_trainer.iit_predict(
    X_base_test, [X_sources_test], 
    intervention_ids_test, 
    id_to_coords, device="cpu"
)

In [240]:
print(classification_report(y_IIT_test, IIT_preds))

              precision    recall  f1-score   support

           2       1.00      0.50      0.67         2
           3       1.00      0.25      0.40        12
           4       0.86      0.46      0.60        13
           5       0.36      0.25      0.30        16
           6       0.51      0.70      0.59        30
           7       0.90      0.63      0.75        30
           8       0.57      0.57      0.57        23
           9       0.61      0.52      0.56        27
          10       0.53      0.66      0.59        38
          11       0.71      0.86      0.77        14
          12       0.55      0.57      0.56        54
          14       0.65      0.74      0.69        23
          15       0.69      0.92      0.79        24
          16       0.62      0.57      0.59        28
          18       0.58      0.62      0.60        58
          20       0.59      0.59      0.59        49
          21       0.67      0.60      0.63        20
          22       0.67    

control setting

In [241]:
control_dataset_fun = get_IIT_arithmetic_dataset_control_V1

In [242]:
X_base_train, y_base_train, X_sources_train, y_IIT_train, intervention_ids_train = \
    control_dataset_fun(
    variable_range, 42, 6400, ab_c
)

/afs/cs.stanford.edu/u/wuzhengx/.local/lib/python3.7/site-packages/ipykernel_launcher.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/afs/cs.stanford.edu/u/wuzhengx/.local/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/afs/cs.stanford.edu/u/wuzhengx/.local/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [243]:
X_base_test, y_base_test, X_sources_test, y_IIT_test, intervention_ids_test = \
    control_dataset_fun(
    variable_range, 66, 1000, ab_c
)

/afs/cs.stanford.edu/u/wuzhengx/.local/lib/python3.7/site-packages/ipykernel_launcher.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/afs/cs.stanford.edu/u/wuzhengx/.local/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/afs/cs.stanford.edu/u/wuzhengx/.local/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [244]:
layer = 0
id_to_coords = {
    2: [{"layer": layer, "start": 0*embedding_dim, "end": 1*embedding_dim}],
    # 2: [{"layer": layer, "start": 1*embedding_dim, "end": 2*embedding_dim}],
}
iit_data = ([X_sources_train], y_IIT_train, intervention_ids_train)

LIM = LIMDeepNeuralClassifier(
    hidden_dim=hidden_dim_per_slice*3, 
    hidden_activation=torch.nn.ReLU(), 
    num_layers=3,
    input_dim=embedding_dim*3,
    n_classes=n_classes,
    device=device,
    vocab_size=(variable_range[1]-variable_range[0]+1),
    embed_dim=embedding_dim,
)

LIM_trainer = LIMTrainer(
    LIM,
    warm_start=True,
    max_iter=500,
    batch_size=6400,
    n_iter_no_change=10000,
    shuffle_train=False,
    eta=0.1,
    save_checkpoint_per_epoch=False,
    input_as_ids=True,
    device=device,
    class2index=class2index,
)

PATH = f"./saved_models_arithmetic/test.bin"
LIM_trainer.model.load_state_dict(torch.load(PATH))
LIM_trainer.model.set_analysis_mode(True)

unfreezing the embedding layer as well.
freezing the embedding layer as well.


In [245]:
_ = LIM_trainer.fit(
    X_base_train, 
    y_base_train, 
    iit_data=iit_data,
    intervention_ids_to_coords=id_to_coords)

Finished epoch 500 of 500; error is 8.179171562194824

In [246]:
IIT_preds = LIM_trainer.iit_predict(
    X_base_test, [X_sources_test], 
    intervention_ids_test, 
    id_to_coords, device="cpu"
)

In [247]:
print(classification_report(y_IIT_test, IIT_preds))

              precision    recall  f1-score   support

           2       0.00      0.00      0.00         2
           3       0.50      0.18      0.27        11
           4       0.50      0.15      0.23        20
           5       0.18      0.12      0.15        16
           6       0.33      0.46      0.38        39
           7       0.33      0.21      0.26        28
           8       0.18      0.23      0.21        30
           9       0.39      0.34      0.37        32
          10       0.26      0.40      0.32        30
          11       0.40      0.22      0.29         9
          12       0.32      0.28      0.30        64
          14       0.42      0.38      0.40        26
          15       0.33      0.28      0.30        25
          16       0.24      0.30      0.27        27
          18       0.29      0.33      0.31        42
          20       0.23      0.23      0.23        44
          21       0.42      0.41      0.41        32
          22       0.00    